$$


In [1]:
import time

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
from ngsolve.krylovspace import CGSolver

In [2]:
n = specialcf.normal(3)
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))
def Curl(u):
    if u.dim == 3:
        return CF( (u[1].Diff(z)- u[2].Diff(y), u[2].Diff(x)- u[0].Diff(z), u[0].Diff(y)- u[1].Diff(x)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )
def Inc(u):
    return Curl((Curl(u)).trans)
P_n = OuterProduct(n,n) 
Q_n = Id(3) - OuterProduct(n,n) 
C_n = CF(      (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) )
def C(V): 
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 



We need to imlement the following with $\sigma$ in $Reg^k$ and $\phi$ in lagrangian of order $k+1$ since the integrals over the Edges have to be well defined and the point evaluation is needed, therefore if $k=0$ then we need some continuity on the vertices!
$$
\sum_T \int_T inc (\sigma) \phi
\quad - \quad
2\sum_F \int_{F} C_n \, curl_f( u ) : \phi 
\quad - \quad
2\sum_F\int_{F} C_n \, rot_f( u ) : \phi 
\quad + \quad
\sum_E\int_{E} t^T( \phi C_n u  -u C_n \phi  ) n 
$$

With $$curl_f( u ) = curl(u Q_n)P_n $$ and  $$ rot_f( u ) = curl(u P_n)Q_n $$


In [3]:
def RunP(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():
        start = time.time()
        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # continuous functions
        peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))

        PEAK = CF ( (peak, 0, 0 , 0,peak,0,0,0,peak), dims=(3,3))
        IncPeak = CF( Inc(PEAK), dims=(3,3) )

        # grid functions 
        gfG = GridFunction(fesHCurlCurl) 
        gfG.Set ( PEAK, bonus_intorder=10, dual=True)

        gfCurlCurlG = GridFunction(fesHCurlCurl)
        gfCurlCurlG.Set(IncPeak, BND,  bonus_intorder=9, dual=True)

        end = time.time()
        print("Time coeff and frid functions :"+str(end-start))
        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################
        start = time.time()
        u,v = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl)
        a += InnerProduct(u,v)*dx 


        # linear form induced by the metric gfG
        f = LinearForm(fesHCurlCurl)

        # thetrahedron inc part
        f += InnerProduct(gfG.Operator("inc"),v)*dx        
        # faces part:
        f += ( InnerProduct(Q_n*n_cross_v, curl(gfG).trans) + Cross(gfG*n,n)*(curl(v)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        f += (gfG[n1,e]*v[e,t1] - gfG[n2,e]*v[e,t2])*dx(element_vb=BBND)
        

        end = time.time()
        print("Time define linear and blinear form :"+str(end-start))

        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################

        start = time.time()
        pre = Preconditioner(a, "local")
        a.Assemble()
        f.Assemble()


        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlCurlG.vec
        inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-9)
        gfCurlCurlG.vec.data += inverse * r
        #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r
        


        end = time.time()
        print("Assemble and find inverse :"+str(end-start))

        start = time.time()
        print("error of my inc")
        Error = norm(IncPeak-gfCurlCurlG,mesh)
        print("Error inc(PEAK)-gfcurlCurlG when order is "+str(Order)+"-->"+str(Error) )
        Draw (gfCurlCurlG-IncPeak,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (gfCurlCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (IncPeak, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 

        end = time.time()
        print("time to draw :"+str(end-start))
        return gfCurlCurlG

In [4]:
Maxh = 0.1
Order = 2
with TaskManager():
    RunP(Maxh, Order)


Time coeff and frid functions :2.2959818840026855
Time define linear and blinear form :0.0019948482513427734
CG converged in 809 iterations to residual 2.3346858790336428e-08
Assemble and find inverse :9.838607549667358
error of my inc
Error inc(PEAK)-gfcurlCurlG when order is 2-->0.09840398208894656


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

time to draw :6.036259651184082


In [5]:
def Run(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():
        start = time.time()
        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # continuous functions
        peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))

        PEAK = CF ( (peak, 0, 0 , 0,peak,0,0,0,peak), dims=(3,3))
        IncPeak = CF( Inc(PEAK), dims=(3,3) )

        # grid functions 
        gfG = GridFunction(fesHCurlCurl) 
        gfG.Set ( PEAK, bonus_intorder=10, dual=True)

        gfCurlCurlG = GridFunction(fesHCurlCurl)
        gfCurlCurlG.Set(IncPeak, BND,  bonus_intorder=9, dual=True)

        end = time.time()
        #print("Time coeff and frid functions :"+str(end-start))
        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################
        start = time.time()
        u,v = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl)
        a += InnerProduct(u,v)*dx 


        # linear form induced by the metric gfG
        f = LinearForm(fesHCurlCurl)

        # thetrahedron inc part
        f += InnerProduct(gfG.Operator("inc"),v)*dx        
        # faces part:
        f += ( InnerProduct(Q_n*n_cross_v, curl(gfG).trans) + Cross(gfG*n,n)*(curl(v)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        f += (gfG[n1,e]*v[e,t1] - gfG[n2,e]*v[e,t2])*dx(element_vb=BBND)
        

        end = time.time()
        #print("Time define linear and blinear form :"+str(end-start))

        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################

        start = time.time()
        #pre = Preconditioner(a, "local")
        a.Assemble()
        f.Assemble()


        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlCurlG.vec
        #inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=500,tol=1e-9)
        gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r
        #gfCurlCurlG.vec.data += inverse * r


        end = time.time()
        #print("Assemble and find inverse :"+str(end-start))

        start = time.time()
        #print("error of my inc")
        Error = norm(IncPeak-gfCurlCurlG,mesh)
        #print("Error inc(PEAK)-gfcurlCurlG when order is "+str(Order)+"-->"+str(Error) )
        #Draw (gfCurlCurlG-IncPeak,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        #Draw (gfCurlCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        #Draw (IncPeak, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 

        end = time.time()
        print("Orrder "+str(Order)+" Meshsize "+str(Maxh))
        return Error

In [6]:
#import matplotlib; import numpy as np; import matplotlib.pyplot as plt; 
#E = []
#for Order in [0,1,2,3,4,5]:
#    E.append(Run(Maxh, Order))
#
#

In [7]:
#print(E)
#plt.plot([0,1,2,3,4,5],np.log(E) )
#plt.grid(True)
#plt.show()

In [8]:
#import matplotlib; import numpy as np; import matplotlib.pyplot as plt; 
#E0 = []
#for Maxh in [0.2,0.1,0.05,0.025]:
#    E0.append(Run(Maxh, 0))
#
#print(E0)
#plt.plot([0.2,0.1,0.05,0.025],np.log(E0) )
#plt.grid(True)
#plt.show()

In [9]:
#import matplotlib; import numpy as np; import matplotlib.pyplot as plt; 
#E1 = []
#for Maxh in [0.2,0.1,0.05,0.025]:
#    E1.append(Run(Maxh, 1))
#
#print(E1)
#plt.plot([0.2,0.1,0.05,0.025],np.log(E1) )
#plt.grid(True)
#plt.show()

In [10]:
#import matplotlib; import numpy as np; import matplotlib.pyplot as plt; 
#E2 = []
#for Maxh in [0.2,0.1,0.05,0.025]:
#    E2.append(Run(Maxh, 2))
#
#print(E1)
#plt.plot([0.2,0.1,0.05,0.025],np.log(E2) )
#plt.grid(True)
#plt.show()